In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
current_path = os.getcwd()
dataset_path = os.path.join(current_path, 'dataset')
dir_list = os.listdir(dataset_path)

In [3]:
dir_list

['SDwarnings_10_09to06_16.csv',
 '.DS_Store',
 'GAPaidSearchTraffic.csv',
 'GAPaidSearchTraffic2.csv',
 'BTSCountryCodes.csv',
 'SDamerican_deaths_abroad_10_09_to_06_16.csv',
 'continents2.csv',
 'GAPaidSearchTraffic2.xlsx',
 'BTSOriginUS_10_09_to_06_16.csv',
 'backup']

<p>get the .csv and .xlsx files into 'dataset' folder. The 'dataset' path is into dataset_path variable</p>

In [3]:
type_converter_dict = {'int64': 'integer',
        'float64': 'real',
        'object': 'character varying', 
        'bool': 'boolean',
        'id': 'serial'}

In [4]:
createSQL = "READ ME: run thin code into PSQL Tool"

In [5]:
for file in dir_list:
    file_path = os.path.join(dataset_path, file)
    schema = file.split('.')[0]
    if '.' not in file:
        break

    ext = file.split('.')[1]
    
    if ext == 'csv':
        df = pd.read_csv(file_path, sep=None, engine="python")
    elif ext == 'xlsx':
        df = pd.read_excel(file_path)

        new_csv_file = "dataset/" + file.split('.')[0] + ".csv"
        
        df.to_csv(new_csv_file,  sep=',', index=False, encoding='utf-8') 

    createSQL = "CREATE TABLE "
    createSQL = createSQL + (schema + "." + file.split('.')[0]).replace(" ", "")
    createSQL = createSQL + "\n(\n"

    for column in df.columns:
        column_name = ''.join(letter for letter in column if letter.isalnum())
        if column_name == "Unnamed0":
            column_name = "id"
        createSQL = createSQL + column_name + " "
        createSQL = createSQL + type_converter_dict[str(df[column].dtypes)] + ",\n"
    
    createSQL = createSQL[:-2]
    createSQL = createSQL + "\n);"
    
    copySQL = "\nCOPY "+ (schema + "." + file.split('.')[0]).replace(" ", "") + " FROM '" + file_path + "' WITH (FORMAT csv);"
    createSQL = createSQL + "\n" + copySQL + "\n\n"

    f = open("create_table.sql", "a")
    f.write(createSQL)
    f.write('')
    f.close()

GAPaidSearchTraffic2.xlsx
